In [1]:
import sqlite3 
import json 

Some explorations to use a database to the people data that we currently have in json

**What this will provide**

The ability to query data that we currently store in json offers to following benefits
- can shuffle the person sequence data with only loading the identifier into memory
- can easily select certain sequences for further processing, for instance for non-MLM encoding for inference on selected subsamples 
- can extend the database with
    - tabular information from raw data 
    - store embedding data as lists or vectors (TBD)
- process chunks of records in parallel, either through python multiprocessing or through slurm parallelization
    - how to re-combine them into one hdf5 file?
    - see [here](https://duckdb.org/docs/guides/python/multiple_threads) for some docs on multi-threaded reading and writing with duckb
    - how does this work with sqlite?

**Open questions**
- this should also work with DuckDB, but we should do some performance tests
- we can store embeddings in different tables, also as jsons
- for training, we can still store the data in hdf5
- how can we fast write during inference? is this fast with sqlite? 
- create some benchmarks?
    - benchmark on speed? 
    - on parallel processing -> but what do we want to parallelize?
    - on reads for downstream tasks
    - on writes for inference
- time complexity
    - there is additional overhead from converting columns to jsons, which is not the case in our current approach 
    - the questions is whether the gain from being able to query the data easily will outweigh the costs
- other considerations 
    - persistent storage? - not really. sqlite is super stable. more an issue for DuckDB. but how much do we care about really long storage?
    - ability to both use all compute available and give results to simple queries on small computers (OSSC work env)
- with this approach, can we also query the dicts from the database? ie, `json_extract` 
- how about writing speed duckdb vs sqlite? ie, when creating a new table, such as the one with the person records?

**Sources**
- Claude.ai
- https://www.beekeeperstudio.io/blog/sqlite-json-with-text
- https://www.sqlite.org/json1.html
- https://berthub.eu/articles/posts/big-data-storage/
- https://blog.brunk.io/posts/similarity-search-with-duckdb/

In [2]:
data = [
    {
        "person_id": 1, 
        "segment": [2, 4, 6],
        "age": [5., 6., 8., 8., 9., 11.],
        "background": {"birth_year": "year_99.0", "birth_month": "month_12.0", "gender": "gender_2.0", "origin": "municipality_54.0"},
        "sentence": [ ["educSim_2.0"], ["_4_D"], ["contractType2_1.0", "sicknessInsurance2_1.0", "wage_50.0"] ]
    },
    {
        "person_id": 2, 
        "segment": [6, 3, 10],
        "age": [10., 1., 5., 3., 4., 16.],
        "background": {"birth_year": "year_95.0", "birth_month": "month_5.0", "gender": "gender_1.0", "origin": "municipality_15.0"},
        "sentence": [ ["educSim_1.0"], ["contractType2_1.0", "_4_D"], ["contractType2_1.0", "wage_50.0", "sicknessInsurance2_2.0"] ]
    }
]

In [3]:

def prepare_person_data(person, columns, json_cols):
    output = []
    for col in columns:
        x = person[col]
        if col in json_cols:
            x = json.dumps(x)
        output.append(x)
    return tuple(output)


def insert_persons(db_conn, persons, columns, json_cols):
    prepared_data = [prepare_person_data(person, columns, json_cols) for person in persons]
    db_conn.executemany("INSERT INTO persons (person_id, segment, age, background, sentence) VALUES (?, ?, ?, ?, ?)", prepared_data)
    db_conn.commit()


In [5]:
dbname = "mydb.sqlite"
db_conn = sqlite3.connect(dbname)


In [6]:
table_name = "persons"
db_conn.execute(f"drop table if exists {table_name}")
db_conn.execute("""CREATE TABLE """ + table_name +
    """ (person_id INT NOT NULL
        , segment TEXT NOT NULL
        , age TEXT NOT NULL
        , background TEXT NOT NULL
        , sentence TEXT NOT NULL
        , PRIMARY KEY ( person_id )
        )""")

In [9]:
text_vars = ["segment", "age", "background", "sentence"]
db_cols = ["person_id", "segment", "age", "background", "sentence"]


In [11]:
insert_persons(db_conn, data, columns=db_cols, json_cols=text_vars)